# LiDAR LAZ Tiling with LAStools

This notebook‑style script shows how to split large LiDAR `.laz` files into smaller, overlapping tiles using the LAStools `lastile` utility. Each section includes explanations and runnable Python code.

---

## Prerequisites

- **Python 3.x**  
- **LAStools** installed, with `lastile` on your system PATH  
- A folder of input `.laz` files to tile  

Run this cell to verify `lastile` is available:


In [18]:
import os
import subprocess

In [19]:
try:
    subprocess.run(["lastile", "-h"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print(" lastile is available!")
except FileNotFoundError:
    print(" lastile not found. Please install LAStools and add to PATH.")

 lastile is available!


# Configuration

This script automates subdivision of reprojected LAZ files into buffered square tiles using the LAStools `lastile` utility. Each tile is written in compressed `.laz` format, ready for downstream processing.

#### 1. Configuration variables

* **INPUT\_DIR**
  Directory containing reprojected `.laz` files.
* **OUTPUT\_DIR**
  Destination for tiled output. Created with `os.makedirs(..., exist_ok=True)`.
* **TILE\_SIZE**
  Width and height of each tile in ground units (meters).
* **BUFFER**
  Padding around each tile to capture edge points, preventing gaps when tiles are processed independently.
* **CORES**
  Number of parallel processes passed to `lastile`. Improves throughput on multicore machines.
* **OUTPUT\_FORMAT**
  Determines use of `-olaz` (compressed) or `-olas` (uncompressed LAS).

#### 2. Workflow

1. **Prepare output folder**

   ```python
   os.makedirs(OUTPUT_DIR, exist_ok=True)
   ```

   Ensures that the script can write tiles without manual directory creation.

2. **Iterate over input files**

   ```python
   for filename in os.listdir(INPUT_DIR):
       if not filename.lower().endswith(".laz"):
           continue
       input_path = os.path.join(INPUT_DIR, filename)
   ```

   Skips non‑LAZ files and constructs the full path for each input.

3. **Build the `lastile` command**

   ```python
   command = [
       "lastile",
       "-i", input_path,
       "-tile_size", str(TILE_SIZE),
       "-buffer", str(BUFFER),
       "-odir", OUTPUT_DIR,
       "-olaz",
       "-cores", str(CORES)
   ]
   ```

   * `-i <input_path>`: input LAZ file
   * `-tile_size <TILE_SIZE>`: tile dimensions
   * `-buffer <BUFFER>`: overlap in meters
   * `-odir <OUTPUT_DIR>`: where to write tiles
   * `-olaz` or `-olas`: output format flag
   * `-cores <CORES>`: parallel execution

4. **Execute and handle errors**

   * `check=True` raises an exception on non‑zero exit codes.
   * Errors for individual files are caught; the loop continues to the next file.

#### 3. Why use this script?

* **Automation**: Eliminates manual typing of `lastile` commands for each file.
* **Consistency**: Ensures that all tiles use the same size, buffer, and compression settings.
* **Scalability**: Processes potentially hundreds of LAZ files in one run, leveraging multiple CPU cores.
* **Integration**: Fits well into the pipeline after reprojection, run this script to prepare data for metric extraction and differencing.

#### Alternative: Direct `lastile` Command

Instead of running the Python wrapper, the same tiling operation can be performed directly in the terminal using the LAStools `lastile` command. This may be preferable if the user already has `lastile` available and prefers shell scripting over Python. For example:

```bash
lastile \
  -i "C:/…/USFS_Tahoe_National_2014_reproj/*.laz" \
  -tile_size 1000 \
  -buffer 20 \
  -odir "C:/…/USFS_Tahoe_National_Tiled_2014" \
  -olaz \
  -cores 4

By running this Python wrapper, users gain a reproducible, error‑resilient tiling step that requires only configuration of a few top‑of‑script variables.

Edit these variables to match your environment and tiling preferences:

In [20]:
import os
import subprocess

# Path to folder containing your input .laz files
INPUT_DIR = r"C:\Users\sreeja\Documents\Tahoe National Park\USFS_Tahoe_National_2014_reproj"

# Path where the tiled files will be written (will be created if needed)
OUTPUT_DIR = r"C:\Users\sreeja\Documents\Tahoe National Park\USFS_Tahoe_National_Tiled_2014"

# Size of each square tile in meters
TILE_SIZE = 1000

# Buffer (overlap) around each tile in meters, to avoid edge effects
BUFFER = 20

# Number of CPU cores to use for parallel tiling
CORES = 4

# Output format: 'laz' (compressed) or 'las' (uncompressed)
OUTPUT_FORMAT = "laz"

# Create the output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output directory ready: {OUTPUT_DIR}")


Output directory ready: C:\Users\sreeja\Documents\Tahoe National Park\USFS_Tahoe_National_Tiled_2014


In [ ]:
for filename in os.listdir(INPUT_DIR):
    if not filename.lower().endswith(".laz"):
        continue

    input_path = os.path.join(INPUT_DIR, filename)
    print(f"Processing {input_path}…")

    command = [
        "lastile",
        "-i",         input_path,        # input file
        "-tile_size", str(TILE_SIZE),     # tile size in meters
        "-buffer",    str(BUFFER),        # buffer in meters
        "-odir",      OUTPUT_DIR,         # output directory
        "-olaz",                           # write compressed LASzip (.laz)
        "-cores",     str(CORES)          # number of parallel cores
    ]

    try:
        subprocess.run(command, check=True)
        print(f" Finished tiling {filename}")
    except subprocess.CalledProcessError as e:
        print(f" Error tiling {filename}: exit code {e.returncode}")
        continue

print(f"\nAll done! Your tiles are in:\n   {OUTPUT_DIR}")

Processing C:\Users\sreeja\Documents\Tahoe National Park\USFS_Tahoe_National_2014_reproj\ot_656000_4370000.laz…
 Finished tiling ot_656000_4370000.laz
Processing C:\Users\sreeja\Documents\Tahoe National Park\USFS_Tahoe_National_2014_reproj\ot_656000_4371000.laz…
 Finished tiling ot_656000_4371000.laz
Processing C:\Users\sreeja\Documents\Tahoe National Park\USFS_Tahoe_National_2014_reproj\ot_657000_4366000.laz…
 Finished tiling ot_657000_4366000.laz
Processing C:\Users\sreeja\Documents\Tahoe National Park\USFS_Tahoe_National_2014_reproj\ot_657000_4367000.laz…
 Finished tiling ot_657000_4367000.laz
Processing C:\Users\sreeja\Documents\Tahoe National Park\USFS_Tahoe_National_2014_reproj\ot_657000_4368000.laz…
 Finished tiling ot_657000_4368000.laz
Processing C:\Users\sreeja\Documents\Tahoe National Park\USFS_Tahoe_National_2014_reproj\ot_657000_4369000.laz…
 Finished tiling ot_657000_4369000.laz
Processing C:\Users\sreeja\Documents\Tahoe National Park\USFS_Tahoe_National_2014_reproj\ot_65